# Chatbot with Memory

Let's add some history to the interaction and build a chatbot. Unlike many people think. LLMs are fixed in their state. They are trained until a certain cutoff date and do not know anything after that point unless you feed them current information. That is also why LLMs do not remember anything about you or the prompts you send to the model. If the model seems to remember you and what you said it is always because the application you are using (e.g. ChatPGT or the chat function in SAP AI Launchpad) is sending the chat history to the model to provide the conversation history to the model as context.

Below you can find a simple implementation of a chatbot with memory.

The code in this exercise is based on the [help documentation](https://help.sap.com/doc/generative-ai-hub-sdk/CLOUD/en-US/_reference/orchestration-service.html) of the Generative AI Hub Python SDK.

In [ ]:
import init_env
import variables
from typing import List

init_env.set_environment_variables()

### Import the packages you want to use

In [ ]:
from gen_ai_hub.orchestration.models.llm import LLM
from gen_ai_hub.orchestration.models.message import Message, SystemMessage, UserMessage
from gen_ai_hub.orchestration.models.template import Template, TemplateValue
from gen_ai_hub.orchestration.models.config import OrchestrationConfig
from gen_ai_hub.orchestration.service import OrchestrationService

### Create the chatbot class

In [ ]:
class ChatBot:
    def __init__(self, orchestration_service: OrchestrationService):
        self.service = orchestration_service
        self.config = OrchestrationConfig(
            template=Template(
                messages=[
                    SystemMessage("You are a helpful chatbot assistant."),
                    UserMessage("{{?user_query}}"),
                ],
            ),
            # TODO add a model name here, e.g. gemini-1.5-flash
            llm=LLM(name=""),
        )
        self.history: List[Message] = []

    def chat(self, user_input):
        response = self.service.run(
            config=self.config,
            template_values=[
                TemplateValue(name="user_query", value=user_input),
            ],
            history=self.history,
        )

        message = response.orchestration_result.choices[0].message

        self.history = response.module_results.templating
        self.history.append(message)

        return message.content
    
    def reset(self):
        self.history = []

service = OrchestrationService(api_url=variables.AICORE_ORCHESTRATION_DEPLOYMENT_URL)
bot = ChatBot(orchestration_service=service)

In [ ]:
bot.chat("Hello, are you there?")

In [ ]:
bot.chat("Do you like CodeJams?")

In [ ]:
bot.chat("Can you remember what we first talked about?")

In [ ]:
bot.history

And to prove to you that the model does indeed not remember you, let's delete the history and try again :)

In [ ]:
bot.reset()
bot.chat("Can you remember what we first talked about?")

[Next exercise - OPTIONAL](11-ai-agents.ipynb)